In [104]:
# Include packages here
import uuid
import hashlib
import random

In [123]:
class Agent:
    def __init__(self):
        self.id = uuid.uuid1()
        self.messagesReceived = []
        self.messagesToSend = []
        self.messagesQueue = []
        self.currency = 0
        
    def sendMessages(self, agents):
        if len(self.messagesToSend) > 0:
            
            # Choose a random number of messages to be sent
            times = 80
            
            for x in range(0,times):
                agent = agents[random.randint(0,len(agents)-1)]
                
                if agent.id != self.id:
                    agent.messagesQueue.append(self.messagesToSend[random.randint(0,len(self.messagesToSend)-1)])
                    
    def analyseQueueMessages(self):
        if len(self.messagesQueue) > 0:
            for message in self.messagesQueue:
                if not any(message.id == x.id for x in self.messagesReceived) and not any(message.fromA == x.fromA for x in self.messagesReceived):
                    self.messagesReceived.append(message)
                    
        self.messagesQueue = []
    
    def checkAmount(self,blockchain,agent):
        check = 0
        for block in blockchain:
            if block.sender == agent:
                check -= block.data
            elif block.receiver == agent:
                check += block.data
    
        return check
    
    def analyseReceivedMessages(self, blockchain):
        if len(self.messagesReceived) > 0:
            
            # Check all messages received ONCE
            for message in self.messagesReceived:
                
                # it means that this agent hasn't checked a specific message
                if message.counter == 0:
                    check = self.checkAmount(blockchain,message.sender)
                    
                    if check < message.data:
                        message.counter = -1
                        print("Invalid Data Found by an Agent :(")
                    else:
                        message.counter = 1
                        print("This data seems to be ok, further analysis needed :)")
            
            toDelete = []
            
            for x in self.messagesReceived:
                for y in self.messagesReceived:
                    
                    if x.id == y.id and x.fromA != y.fromA:
                        for z in blockchain:
                            if z.id == x.id:
                                z.counter += x.counter
                                self.messagesReceived.remove(x)
                                self.messagesReceived.remove(y)
                                              
    def checkBlockchainQueue(self, pending):
        
        for aMessage in self.messagesReceived:
            counter = 0
            
            for bMessage in pending:
                if aMessage.id == bMessage.id and aMessage.fromA == bMessage.fromA:
                    counter += 1
                    
            if counter == 0:
                self.messagesReceived.remove(aMessage)

In [115]:
class Block:
    def __init__(self, blockNo, pHash, data, sender, receiver):
        self.blockNo = blockNo
        self.pHash = pHash
        self.data = data
        self.sender = sender
        self.receiver = receiver
        
        self.cHash = self.computeHash(blockNo, pHash, data, sender, receiver)
        
    def computeHash(self,blockNo, pHash, data, sender, receiver):
        string = str(blockNo)+str(pHash)+str(data)+str(sender)+str(receiver)
        encoded = string.encode()
        result = hashlib.sha256(encoded)

        return result.hexdigest()

In [116]:
class pendingBlock:
    def __init__(self, data, sender, receiver, sentBy, ID):
        self.id = ID
        self.sender = sender
        self.receiver = receiver
        self.data = data
        self.fromA = sentBy
        self.checkedBy = []
        self.counter = 0

In [117]:
class BlockChain:
    
    def __init__(self):
        self.blockchain = [self.genesisBlock()]
        self.pendingBlocks = []
        self.agents = []
        
        for x in range(0,100):
            self.newAgentInNetwork()
        
    def genesisBlock(self):
        return Block(0,0,0,0,0)
    
    def addNewBlock(self,newBlock):
        receiver = next((x for x in self.agents if x.id == newBlock.receiver), None)
        sender = next((x for x in self.agents if x.id == newBlock.sender), None)
        
        if newBlock.sender == "Blockchain":
            self.blockchain.append(newBlock)
            
            if receiver != None:
                receiver.currency += newBlock.data

            if sender != None:
                sender.currency -= newBlock.data
                
            print("New Block Entered Successfully! (new agent added)")
            
        elif newBlock.sender != "Blockchain" and sender.currency - newBlock.data > 0:
            self.blockchain.append(newBlock)
            
            if receiver != None:
                receiver.currency += newBlock.data

            if sender != None:
                sender.currency -= newBlock.data
        
            print("New Block (with valid data) entered successfully!")
            
        elif newBlock.sender != "Blockchain" and sender.currency - newBlock.data < 0:
            print("Block rejected! Invalid data!")
            
    
    def newAgentInNetwork(self):
        agent = Agent()
        self.agents.append(agent)
        prevBlock = self.blockchain[len(self.blockchain)-1]
        
        block = Block(len(self.blockchain),
                     prevBlock.cHash,
                     10000,
                     "Blockchain",
                     agent.id)
        
        self.addNewBlock(block)
        
    def checkBlockchainValidity(self):
        counter = 0
        for x in range(len(self.blockchain)):
            if x > 0 and self.blockchain[x].pHash != self.blockchain[x-1].cHash:
                counter += 1
                
        if counter != 0:
            print("Blockchain Validity Check failed!!!")
            
    def checkPending(self):
        for aMessage in self.pendingBlocks:
            for bMessage in self.pendingBlocks:
                
                if aMessage.id == bMessage.id and aMessage.fromA != bMessage.fromA and aMessage.counter > 5 and aMessage.counter == bMessage.counter and aMessage.data == bMessage.data:
                    
                    prevBlock = self.blockchain[len(self.blockchain)-1]
                    block = Block(len(self.blockchain),
                                 prevBlock.cHash,
                                 aMessage.data,
                                 aMessage.sender,
                                 aMessage.receiver)
                    self.addNewBlock(block)
                    self.pendingBlocks.remove(aMessage)
                    self.pendingBlocks.remove(bMessage)
                elif aMessage.id == bMessage.id and aMessage.fromA != bMessage.fromA and aMessage.counter < -5 and aMessage.counter == bMessage.counter and aMessage.data == bMessage.data:
                    self.pendingBlocks.remove(aMessage)
                    self.pendingBlocks.remove(bMessage)
                    
    def step(self, no):
        
        for noOfSteps in range(0,no):
            sender = self.agents[random.randint(0,len(self.agents)-1)]
            receiver = self.agents[random.randint(0,len(self.agents)-1)]

            # Making sure that the sender is not going to send messages to itself
            while sender == receiver:
                receiver = self.agents[random.randint(0,len(self.agents)-1)]

            difMsg = random.randint(1,100)
            invalData = random.randint(1,100)

            # Add a chance that the data sent by sender to be invalid (False)
            if invalData <= 10:
                dataSender = random.randint(1,int(1.1*sender.currency))
            else:
                dataSender = random.randint(1,sender.currency)

            # Add a chance that they are sending different messages
            if difMsg <= 10:
                dataReceiver = random.randint(1,sender.currency)
            else:
                dataReceiver = dataSender


            # Add a chance to have a new transaction
            transaction = random.randint(1,100)
            if transaction <= 30:
                uniqueID = uuid.uuid1()
                messageSender = pendingBlock(dataSender, sender.id, receiver.id, sender.id, uniqueID)
                messageReceiver = pendingBlock(dataReceiver, sender.id, receiver.id, receiver.id, uniqueID)
                sender.messagesToSend.append(messageSender)
                receiver.messagesToSend.append(messageReceiver)
                self.pendingBlocks.append(messageSender)
                self.pendingBlocks.append(messageReceiver)

            for agent in self.agents:
                agent.sendMessages(self.agents)

            for agent in self.agents:
                agent.analyseQueueMessages()

            for agent in self.agents:
                agent.analyseReceivedMessages(self.blockchain)

            for agent in self.agents:
                agent.checkBlockchainQueue(self.pendingBlocks)

            self.checkPending()
            self.checkBlockchainValidity()

In [125]:
blockchain = BlockChain()

New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)
New Block Entered Successfully! (new agent added)


In [ ]:
blockchain.step(10000)

This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis needed :)
This data seems to be ok, further analysis nee